In [1]:
#Import all Required Data Sets and packages

from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import numpy as np
from collections import Counter
import random
import pdb

In [2]:
# Load the Iris dataset
iris = load_iris()

# Print the feature names
#print("Feature names:", iris.feature_names)

# Print the target names
#print("Target names:", iris.target_names)

# Print the first five rows of the data
#print("Data:\n", iris.data[:5])

# Print the first five target values
#print("Target:\n", iris.target[:5])

# Print the shape of the data
print("Data Shape:", np.shape(iris.data))

Data Shape: (150, 4)


In [3]:
# Load the Wine dataset
wine = load_wine()

# Print the feature names
#print("Feature names:", wine.feature_names)

# Print the target names
#print("Target names:", wine.target_names)

# Print the first five rows of the data
#print("Data:\n", wine.data[:5])

# Print the first five target values
#print("Target:\n", wine.target[:5])

# Print the shape of the data
print("Data Shape:", np.shape(wine.data))

Data Shape: (178, 13)


In [4]:
# Load the digits dataset
digits = load_digits()

# Print the shape of the data
print("Data Shape:", np.shape(digits.data))

Data Shape: (1797, 64)


In [5]:
#Class for a node on a tree

class TreeNode:
    #Constructor for a tree node. 
    #p must be the parent node (Tree Node)
    #lc corresponds to the left child of the node (Tree Node)
    #rc corresponds to the right child of the node (Tree Node)
    #If the node is not a leaf node, f corresponds to the feature that the node will split on (int)
    #If the node is not a leaf node, v corresponds to the value that the node will compare (float)
    #If the node is a leaf node, c determines the class that the node ouputs (int)
    #ln is a boolean variable which is true if the node is a leaf node (boolean)
    
    def __init__(self,p = None, lc = None, rc = None, f = None, v = None, c = None, ln = None):
        #Stores the parent node if needed
        self.parent = p
        #Stores left child
        self.leftChild = lc
        #Stores right child
        self.rightChild = rc
        #Stores feature to check
        self.feature = f
        #Stores value to split on
        self.value = v
        #Stores classsification to return
        self.classification = c
        #Stores whether or not the node is a leaf node
        self.isLeafNode = ln
    #Setter methods for a variety of properties
    def set_parent(self,p):
        self.parent = p
    def set_left_child(self,lc):
        self.leftChild = lc
    def set_right_child(self,rc):
        self.rightChild = rc
    def set_feature(self, f):
        self.feature = f
    def set_value(self, v):
        self.value = v
    def set_classification(self, c):
        self.classification = c
    #Returns true if node is a leaf node, false otherwise
    def get_is_leaf_node(self):
        return self.isLeafNode
    #If the node is a leaf node, returns the class the node represents
    def get_classification(self):
        if(self.isLeafNode):
            return self.classification
        else:
            return None
    #If the node is not a leaf node, will return the next node in the tree for a specific observation
    def bubble_down(self, observation):
        if(not self.isLeafNode):
            if(observation[self.feature] < self.value):
                return self.leftChild
            else:
                return self.rightChild
        else:
            return None
    #Getter Methods for various properties
    def get_parent(self):
        return self.parent
    def get_right_child(self):
        return self.rightChild
    def get_left_child(self):
        return self.rightChild
    def get_value(self):
        return self.value
    def get_feature(self):
        return self.feature
    #Checks if a value goes to the left child or not
    def goes_left(self, observation):
        if(not self.isLeafNode):
            if(observation[self.feature] < self.value):
                return True
            else:
                return False
        else:
            return None

In [6]:
#Class for a whole tree

class DecisionTree:
    
    #If data and target are specified, builds a decision tree for that data and target
    #r is the root node. If data is provided no root node is needed (TreeNode)
    #md is the max depth the tree should reach. Default 0. (int)
    #data is a 2-dimensional array with each row corresponding to an observation (2d_array)
    #target is the class corresponding to the row in data. i.e. the tree should classify data[i] as target[i] (1d array)
    #ms is the min samples that should be given to a leaf node. Default 1. (int)
    
    def __init__(self, r = None, md = 0, data = None, target = None, ms = 1):
        #sets the max depth
        self.maxDepth = md
        #sets the min samples
        self.minSamples = ms
        #if data and target are not provided, nothing is constructed
        if(data is None or target is None):
            self.root = r
        #if data and target are provided, builds decision tree
        else:
            self.root = self.build_tree(data,target)
    
    #Classifies an observation
    def classify(self, observation):
        #start at the root node
        node = self.root
        #bubble down the tree until a leaf node is reached
        while(not node.get_is_leaf_node()):
            node = node.bubble_down(observation)
        #Classification of leaf node will be what the tree decided the class is
        return node.get_classification()
    #Revursive function to build the tree
    def build_tree(self, data, target, depth = 0):
        #If all the data has the same class, then the node doesn't have to split anymore and it becomes a leaf node
        if(self.check_single_class(target)):
            #Returns leaf node with classification as the only class in the sample
            return TreeNode(ln = True, c = target[0])
        #If max depth is reached, make a leaf node which classifies as the majority class
        elif(depth == self.maxDepth):
            return TreeNode(ln = True, c = self.get_most_probable_class(target))
        #Calculates the best feature and value to split on and then generates two new nodes giving split data
        #and target to the child nodes
        else:
            #Calculates the parent entropy
            parent_entropy = self.get_entropy(target)
            #Gets the best feature and value to split on
            (feature,value) = self.get_feature_split(data,target,parent_entropy)
            #If value was not assigned, there was no split that satisfied the min samples requirement
            #and a leaf node is created.
            if(value is None):
                return TreeNode(ln = True, c = self.get_most_probable_class(target))
            #Generates the new node in the tree
            new_node = TreeNode(f = feature, v = value, ln = False)
            #Splits the data based on the best feature and value found
            (data1, target1, data2, target2) = self.split_data(new_node,data,target)
            #Gives the data and target that go left to the build tree function,
            #The returned node will be the left child
            new_node.set_left_child(self.build_tree(data1,target1,depth = depth + 1))
            #Gives the data and target that go right to the build tree function,
            #The returned node will be the right child
            new_node.set_right_child(self.build_tree(data2,target2,depth = depth + 1))
            #Returns the node generated
            return new_node
    #Determines the best feature and value to split on
    def get_feature_split(self,data,target,parent_entropy):
        #The max information gained starts as smallest possible value
        max_information_gain = float("-inf")
        #No feature or value assigned yet
        feature = None
        value = None
        #Iterates over all observations
        for i in range(len(data)):
            #Iterates over the features
            for j in range(len(data[i])):
                #Assigns the current feature
                feature_split = j
                #Asssigns the current observations value
                feature_value = data[i][j]
                #Create a new node to split on the value
                node = TreeNode(ln=False,f = feature_split, v = feature_value)
                #Split the data
                (data1,target1,data2,target2) = self.split_data(node,data,target)
                #Check that the split gives at least the min samlples to the next node
                if(len(target1) >= self.minSamples and len(target2) >= self.minSamples):
                    #Computes the entropy in the left split
                    left_entropy = self.get_entropy(target1)
                    #Computes the entropy in the right split
                    right_entropy = self.get_entropy(target2)
                    #Computes the information gained with the current split
                    information_gain = parent_entropy - (len(target1)*left_entropy+len(target2)*right_entropy)/len(target)
                    #If this is the most information gained thus far, then split on this feature and this value
                    if(information_gain > max_information_gain):
                        feature = feature_split
                        value = feature_value
        #Returns feature and value that gives the most information gain
        return (feature, value)
    #Checks if the traget only has a single class in it
    def check_single_class(self, target):
        if(len(np.unique(target)) == 1):
            return True
        else:
            return False
    #Splits the data according to how a node would
    def split_data(self, node,data,target):
        #Data that goes left
        data1 = []
        #Data that goes right
        data2 = []
        #Target that goes left
        target1 = []
        #Target that goes right
        target2 = []
        #Iterates over the data
        for i in range(len(data)):
            left = node.goes_left(data[i])
            #If data goes left add it to the left data array and the target as well
            if(left):
                data1.append(data[i])
                target1.append(target[i])
            #Otherwise data goes right and is added to the data that will go to the right
            else:
                data2.append(data[i])
                target2.append(target[i])
        #Returns the split data
        return(np.array(data1),np.array(target1),np.array(data2),np.array(target2))
    #Computes the most probable class
    def get_most_probable_class(self,target):
        data = Counter(target)
        return data.most_common(1)[0][0]
    #Computes the entropy in a target sample
    def get_entropy(self, target):
        # Calculate the proportion of samples in each class
        p_i = np.unique(target,return_counts = True)[1]/len(target)
        entropy = 0
        # Calculate the entropy according to entropy = sum over all classes i (p_i*log_2(p_i))
        for p in p_i:
            if(p != 0):
                entropy -= p * np.log2(p)
        #returns the entropy
        return entropy


In [7]:
#Builds a random forest

class RandomForest:
    #nt is the number of trees in the forest (int)
    #md is the max depth for each of the trees (int)
    #ms is the min samples that a leaf needs to have (int)
    #data is an array of observations (2d-array)
    #target are the corresponding classes to each observation (1d-array)
    
    def __init__(self,nt,md,ms,data,target):
        #Gets the number of features each tree should consider
        num_features = int(np.sqrt(np.shape(data)[1]))
        #Generates a list of the trees in the forest
        self.tree_list = [None]*nt
        #Keeps track of the features each tree will split on
        self.tree_features = [None]*nt
        #Make trees equal to the trees in the forest
        for i in range(nt):
            #Gets random features to consider and random data
            (tree_data,tree_target,tree_features) = self.GetRandomFeatureData(data,target,num_features)
            #Keeps track of the tree features
            self.tree_features[i] = tree_features
            #Makes a new tree
            self.tree_list[i] = DecisionTree(md = md, ms=ms, data = tree_data, target = tree_target)
        return
    #Gets random data for each tree
    def GetRandomFeatureData(self,data,target,num_features):
        #Get a random subset of features this tree will consider
        random_features = [random.randint(0, np.shape(data)[1]-1) for _ in range(num_features)]
        #Sort it and make it unique
        unique_sorted_features = sorted(set(random_features))
        #Generate a random list of indices to pull from data
        random_data = [random.randint(0, np.shape(data)[0]-1) for _ in range(np.shape(data)[0])]
        #Repeated data will only increase time, but won't make the tree better probably
        random_data = sorted(set(random_data))
        #Array for the random data
        tree_data = []
        tree_target = []
        #For each observation to consider add it to the data
        for i in range(len(random_data)):
            row = []
            #Only add the features to the row that will be considered by the tree
            for j in range(len(unique_sorted_features)):
                row.append(data[random_data[i]][unique_sorted_features[j]])
            #Append data
            tree_data.append(row)
            tree_target.append(target[random_data[i]])                
        return(tree_data,tree_target,unique_sorted_features)
    #Classifies an observation by majority vote
    def classify(self,observation):
        #List of classifications
        classifications = []
        #Have each tree classify the observation
        for i in range(len(self.tree_list)):
            sample = []
            #Get the features that the tree considers
            for j in range(len(self.tree_features[i])):
                sample.append(observation[self.tree_features[i][j]])
            classifications.append(self.tree_list[i].classify(sample))
        #Return most common class
        top_class = Counter(classifications)
        return top_class.most_common(1)[0][0]
        

In [8]:
#Small test that the DecisionTree is working
leftleaf = TreeNode(ln = True, c = 0)
rightleaf = TreeNode(ln = True, c = 1)
rootnode = TreeNode(lc = leftleaf, rc = rightleaf, f = 0, v = 0.5, ln = False)
leftleaf.set_parent(rootnode)
rightleaf.set_parent(rootnode)

tree = DecisionTree(rootnode)

print(tree.classify([1000]))

1


In [9]:
#Iris dataset test to test single decision tree
X_train, X_val, y_train, y_val = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# X and y are your feature and target arrays, respectively.
# test_size specifies the proportion of the data that should be allocated to the validation set.
# random_state is used to ensure that the split is reproducible.

tree = DecisionTree(md = 14, data = X_train, target = y_train)

num_correct = 0
num_incorrect = 0

for i in range(len(X_val)):
    output = tree.classify(X_val[i])
    if(output == y_val[i]):
        num_correct += 1
    else:
        num_incorrect += 1
print("Accuracy: ", num_correct/len(y_val))
print("Correct: ", num_correct)
print("Incorrect: ", num_incorrect)

Accuracy:  0.9666666666666667
Correct:  29
Incorrect:  1


In [10]:
#Wine Dataset test to test single decision tree
X_train, X_val, y_train, y_val = train_test_split(wine.data, wine.target, test_size=0.2, random_state=42)

# X and y are your feature and target arrays, respectively.
# test_size specifies the proportion of the data that should be allocated to the validation set.
# random_state is used to ensure that the split is reproducible.

tree = DecisionTree(md = 20, data = X_train, target = y_train)

num_correct = 0
num_incorrect = 0

for i in range(len(X_val)):
    output = tree.classify(X_val[i])
    if(output == y_val[i]):
        num_correct += 1
    else:
        num_incorrect += 1
print("Accuracy: ", num_correct/len(y_val))
print("Correct: ", num_correct)
print("Incorrect: ", num_incorrect)

Accuracy:  0.8333333333333334
Correct:  30
Incorrect:  6


In [11]:
#Digits Dataset test to test single decision tree
X_train, X_val, y_train, y_val = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)

# X and y are your feature and target arrays, respectively.
# test_size specifies the proportion of the data that should be allocated to the validation set.
# random_state is used to ensure that the split is reproducible.

tree = DecisionTree(md = 5, data = X_train, target = y_train)

num_correct = 0
num_incorrect = 0

for i in range(len(X_val)):
    output = tree.classify(X_val[i])
    if(output == y_val[i]):
        num_correct += 1
    else:
        num_incorrect += 1
print("Accuracy: ", num_correct/len(y_val))
print("Correct: ", num_correct)
print("Incorrect: ", num_incorrect)

Accuracy:  0.37777777777777777
Correct:  136
Incorrect:  224


In [12]:
#Iris dataset to test random forest

X_train, X_val, y_train, y_val = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)

# X and y are your feature and target arrays, respectively.
# test_size specifies the proportion of the data that should be allocated to the validation set.
# random_state is used to ensure that the split is reproducible.

random_forest = RandomForest(data = X_train, nt = 50, md = 4, ms = 1, target = y_train)

num_correct = 0
num_incorrect = 0

for i in range(len(X_val)):
    output = random_forest.classify(X_val[i])
    if(output == y_val[i]):
        num_correct += 1
    else:
        num_incorrect += 1
print("Accuracy: ", num_correct/len(y_val))
print("Correct: ", num_correct)
print("Incorrect: ", num_incorrect)

Accuracy:  1.0
Correct:  30
Incorrect:  0


In [13]:
#Wine datset to test RandomForest

X_train, X_val, y_train, y_val = train_test_split(wine.data, wine.target, test_size=0.2, random_state=42)

# X and y are your feature and target arrays, respectively.
# test_size specifies the proportion of the data that should be allocated to the validation set.
# random_state is used to ensure that the split is reproducible.

random_forest = RandomForest(data = X_train, nt = 50, md = 4, ms = 1, target = y_train)

num_correct = 0
num_incorrect = 0

for i in range(len(X_val)):
    output = random_forest.classify(X_val[i])
    if(output == y_val[i]):
        num_correct += 1
    else:
        num_incorrect += 1
print("Accuracy: ", num_correct/len(y_val))
print("Correct: ", num_correct)
print("Incorrect: ", num_incorrect)

Accuracy:  0.8888888888888888
Correct:  32
Incorrect:  4


In [14]:
#Digits dataset to test RandomForest

X_train, X_val, y_train, y_val = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)

# X and y are your feature and target arrays, respectively.
# test_size specifies the proportion of the data that should be allocated to the validation set.
# random_state is used to ensure that the split is reproducible.

random_forest = RandomForest(data = X_train, nt = 100, md = 4, ms = 1, target = y_train)

num_correct = 0
num_incorrect = 0

for i in range(len(X_val)):
    output = random_forest.classify(X_val[i])
    if(output == y_val[i]):
        num_correct += 1
    else:
        num_incorrect += 1
print("Accuracy: ", num_correct/len(y_val))
print("Correct: ", num_correct)
print("Incorrect: ", num_incorrect)

Accuracy:  0.6916666666666667
Correct:  249
Incorrect:  111
